# Introduction

QA plots for the INTT detector

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for newer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for newer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

via sPHENIX software distribution at `/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_ana/ana.470`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery.git

## JenkinsCI information (if available)

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [9]:
%jsroot on

In [10]:

qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))


`qa_file_name_new` env not set. use the default `qa_file_name_new=prdf_reconstruction53877_qa.root`

In [11]:
# qa_file_new = ROOT.TFile.Open(qa_file_name_new);

# assert qa_file_new.IsOpen()
# qa_file_new.ls()
display(Markdown(f"Openning QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

Openning QA file at `prdf_reconstruction53877_qa.root`

22620660404224

In [12]:
%%cpp

if (qa_file_new == nullptr) 
{
    cout <<"Error, can not open QA root file"<<qa_file_name_new<<endl;
    exit(1);
}
// list inputs histograms if needed
 //qa_file_new ->ls();

In [13]:
%%cpp

void MVTXDrawChipInfo(TFile* qa_file_new) {
    std::cout << "MVTX DrawChipInfo() Beginning" << std::endl;
    
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error: Invalid QA root file provided." << std::endl;
        return;
    }

    std::string histprefix = "h_MvtxClusterQA_";

    // Get histograms from the file using the prefix and dynamic_cast

    TH1F *h_occupancy = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "chipOccupancy").c_str()));
    TH1F *h_clusSize = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusterSize").c_str()));
    TH1I *h_strobe = dynamic_cast<TH1I*>(qa_file_new->Get((histprefix + "strobeTiming").c_str()));

    // Create a canvas for plotting
    TCanvas *canvas = new TCanvas("chip_info", "MVTX Chip Info", 1000, 800);
    canvas->Divide(3, 1);

    // Plot the first histogram (chipOccupancy)
    if (h_occupancy) {
        canvas->cd(1);
        h_occupancy->SetTitle("MVTX Chip Occupancy");
        h_occupancy->SetXTitle("Chip Occupancy [%]");
        h_occupancy->SetYTitle("Normalized Entries");
        //h_occupancy->GetXaxis()->SetRange(1, h_occupancy->GetNbinsX() + 1); // include overflow bin
        h_occupancy->Scale(1. / h_occupancy->Integral());  // Normalize
        h_occupancy->Draw();
        gPad->SetLogy();  // Logarithmic scale for y-axis
        gPad->SetRightMargin(0.15);
    } else {
        std::cerr << "Histogram chipOccupancy not found" << std::endl;
    }

    // Plot the second histogram (clusterSize)
    if (h_clusSize) {
        canvas->cd(2);
        h_clusSize->SetTitle("MVTX Cluster Size");
        h_clusSize->SetXTitle("Cluster Size");
        h_clusSize->SetYTitle("Normalized Entries");
        //h_clusSize->GetXaxis()->SetRange(1, h_occupancy->GetNbinsX() + 1); // include over flow bin
        h_clusSize->Scale(1. / h_clusSize->Integral());  // Normalize
        h_clusSize->Draw();
        gPad->SetRightMargin(0.15);
    } else {
        std::cerr << "Histogram clusterSize not found" << std::endl;
    }

    // Plot the third histogram (strobeTiming)
    if (h_strobe) {
        canvas->cd(3);
        h_strobe->Draw();
        gPad->SetRightMargin(0.15);
    } else {
        std::cerr << "Histogram strobe Timing not found" << std::endl;
    }

    // Add run information text to the canvas
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.04);
    PrintRun.SetNDC();  // Set to normalized coordinates
    PrintRun.SetTextAlign(23);  // Center alignment

    // Create the run information string
    std::ostringstream runnostream1;
    std::string runstring1;
    //runnostream1 << cl->Name() << "_mvtx Info Run " << cl->RunNumber() << ", build " << cl->build();
    runstring1 = runnostream1.str();

    // Draw the run information text in the top-center of the canvas
    canvas->cd(1);  // Use the first pad (or choose a pad based on your needs)
    PrintRun.DrawText(0.5, 1., runstring1.c_str());

    // Update the canvas with the plotted histograms
    canvas->Update();
    std::cout << "MVTX DrawChipInfo() Ending" << std::endl;
}




In [14]:
%%cpp 
MVTXDrawChipInfo(qa_file_new);

MVTX DrawChipInfo() Beginning
MVTX DrawChipInfo() Ending


## Residuals

In [15]:
%%cpp
void MVTXDrawClusterInfo(TFile* qa_file_new)
{
    std::cout << "MVTX DrawClusterInfo() Beginning" << std::endl;

    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return;
    }

    // Define the histogram prefix
    std::string histprefix = "h_MvtxClusterQA_";

    // Get histograms from the file
    TH1F *h_clusPhi_incl = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusterPhi_incl").c_str()));
    TH1F *h_clusPhi_l0 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusterPhi_l0").c_str()));
    TH1F *h_clusPhi_l1 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusterPhi_l1").c_str()));
    TH1F *h_clusPhi_l2 = dynamic_cast<TH1F*>(qa_file_new->Get((histprefix + "clusterPhi_l2").c_str()));
    TH2F *h_clusZ_clusPhi_l0 = dynamic_cast<TH2F*>(qa_file_new->Get((histprefix + "clusterZ_clusPhi_l0").c_str()));
    TH2F *h_clusZ_clusPhi_l1 = dynamic_cast<TH2F*>(qa_file_new->Get((histprefix + "clusterZ_clusPhi_l1").c_str()));
    TH2F *h_clusZ_clusPhi_l2 = dynamic_cast<TH2F*>(qa_file_new->Get((histprefix + "clusterZ_clusPhi_l2").c_str()));

    // Create the canvas (check if it exists, otherwise create it)
    TCanvas* canvas = (TCanvas*)gROOT->FindObject("cluster_info");
    if (!canvas) {
        canvas = new TCanvas("cluster_info", "MVTX Cluster Information", 1000, 800); // 1000x800 size for the canvas
    }
    canvas->Divide(4, 2); // Create a 4x2 grid of pads (4 columns and 2 rows)

    // Create an array for the pads (2 rows, 4 columns)
    TPad* Pad[2][4]; // 2 rows, 4 columns
    for (int i = 0; i < 2; i++) {
        for (int j = 0; j < 4; j++) {
            // Calculate the position of the pads: Each pad will take up 1/4th of the width and 1/2 of the height
            float xLow = 0.25 * j;
            float xHigh = 0.25 * (j + 1);
            float yLow = 0.5 * i;
            float yHigh = 0.5 * (i + 1);
            
            Pad[i][j] = new TPad(Form("Pad_%d_%d", i, j), "", xLow, yLow, xHigh, yHigh);
            Pad[i][j]->Draw();
        }
    }

    // Plot the first histogram (clusterPhi_incl, clusterPhi_l0, clusterPhi_l1, clusterPhi_l2)
    Pad[0][0]->cd();
    if (h_clusPhi_incl && h_clusPhi_l0 && h_clusPhi_l1 && h_clusPhi_l2) {
        h_clusPhi_incl->SetTitle("MVTX Cluster #phi");
        h_clusPhi_incl->SetXTitle("Cluster #phi wrt origin [rad]");
        h_clusPhi_incl->SetYTitle("Entries");
        h_clusPhi_incl->SetMarkerSize(0.5);
        h_clusPhi_incl->Draw();
        h_clusPhi_l0->SetMarkerSize(0.5);
        h_clusPhi_l0->SetMarkerColor(kRed);
        h_clusPhi_l0->SetLineColor(kRed);
        h_clusPhi_l0->Draw("same");
        h_clusPhi_l1->SetMarkerSize(0.5);
        h_clusPhi_l1->SetMarkerColor(kBlue);
        h_clusPhi_l1->SetLineColor(kBlue);
        h_clusPhi_l1->Draw("same");
        h_clusPhi_l2->SetMarkerSize(0.5);
        h_clusPhi_l2->SetMarkerColor(kGreen + 2);
        h_clusPhi_l2->SetLineColor(kGreen + 2);
        h_clusPhi_l2->Draw("same");
        auto legend = new TLegend(0.45, 0.7, 0.7, 0.9);
        legend->AddEntry(h_clusPhi_incl, "Inclusive", "pl");
        legend->AddEntry(h_clusPhi_l0, "Layer 0", "pl");
        legend->AddEntry(h_clusPhi_l1, "Layer 1", "pl");
        legend->AddEntry(h_clusPhi_l2, "Layer 2", "pl");
        legend->SetFillStyle(0);
        legend->Draw();
        gPad->SetRightMargin(0.15);
    } else {
        std::cerr << "Error: Histograms not found!" << std::endl;
        return; // Just exit the function
    }

    // Plot the second histogram (clusterZ_clusPhi_l0)
    Pad[0][1]->cd();
    if (h_clusZ_clusPhi_l0) {
        h_clusZ_clusPhi_l0->SetTitle("MVTX Cluster Z vs phi Layer 0");
        h_clusZ_clusPhi_l0->SetXTitle("Cluster (layer 0) Z [cm]");
        h_clusZ_clusPhi_l0->SetYTitle("Cluster (layer 0) #phi wrt origin [rad]");
        h_clusZ_clusPhi_l0->SetZTitle("Entries");
        h_clusZ_clusPhi_l0->Draw("colz");
        gPad->SetRightMargin(0.17);
        gPad->SetLogz();
    } else {
        std::cerr << "Error: Histogram clusterZ_clusPhi_l0 not found!" << std::endl;
        return; // Just exit the function
    }

    // Plot the third histogram (clusterZ_clusPhi_l1)
    Pad[0][2]->cd();
    if (h_clusZ_clusPhi_l1) {
        h_clusZ_clusPhi_l1->SetTitle("MVTX Cluster Z vs phi Layer 1");
        h_clusZ_clusPhi_l1->SetXTitle("Cluster (layer 1) Z [cm]");
        h_clusZ_clusPhi_l1->SetYTitle("Cluster (layer 1) #phi wrt origin [rad]");
        h_clusZ_clusPhi_l1->SetZTitle("Entries");
        h_clusZ_clusPhi_l1->Draw("colz");
        gPad->SetRightMargin(0.17);
        gPad->SetLogz();
    } else {
        std::cerr << "Error: Histogram clusterZ_clusPhi_l1 not found!" << std::endl;
        return; // Just exit the function
    }

    // Plot the fourth histogram (clusterZ_clusPhi_l2)
    Pad[0][3]->cd();
    if (h_clusZ_clusPhi_l2) {
        h_clusZ_clusPhi_l2->SetTitle("MVTX Cluster Z vs phi Layer 2");
        h_clusZ_clusPhi_l2->SetXTitle("Cluster (layer 2) Z [cm]");
        h_clusZ_clusPhi_l2->SetYTitle("Cluster (layer 2) #phi wrt origin [rad]");
        h_clusZ_clusPhi_l2->SetZTitle("Entries");
        h_clusZ_clusPhi_l2->Draw("colz");
        gPad->SetRightMargin(0.17);
        gPad->SetLogz();
    } else {
        std::cerr << "Error: Histogram clusterZ_clusPhi_l2 not found!" << std::endl;
        return; // Just exit the function
    }

    // Add run information text to the canvas
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.04);
    PrintRun.SetNDC();  // Set to normalized coordinates
    PrintRun.SetTextAlign(23);  // Center alignment

    // Create the run information string
    std::ostringstream runnostream1;
    std::string runstring1;
    // Add run number and other relevant info to the string
    runstring1 = runnostream1.str();

    // Draw the run information text in the top-center of the canvas
    canvas->cd();
    PrintRun.DrawText(0.5, 0.95, runstring1.c_str());

    // Update the canvas with the plotted histograms and text
    canvas->Update();

    std::cout << "DrawClusterInfo Ending" << std::endl;
}


## Errors 

In [16]:
%%cpp 
MVTXDrawClusterInfo(qa_file_new);

MVTX DrawClusterInfo() Beginning
DrawClusterInfo Ending


In [17]:
%%cpp 

void MVTXDrawSummaryInfo(TFile* qa_file_new) {
    std::cout << "MVTX DrawSummaryInfo() Beginning" << std::endl;

    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return;
    }
    
    // Define histprefix (if needed)
    std::string histprefix = "h_MvtxClusterQA_"; // Adjust as needed

    // Retrieve histograms using Get()
    TH1F *h_clusPhi_incl = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "clusterPhi_incl").c_str()));
    TH1F *h_occupancy = dynamic_cast<TH1F *>(qa_file_new->Get((histprefix + "chipOccupancy").c_str()));

    // Create canvas
    TCanvas *canvas = new TCanvas("summary_info", "MVTX Summary Info", 1000, 800);
    canvas->Divide(2, 2);

    // Fit the cluster phi histogram
    TF1 *f1 = new TF1("f1", "pol0(0)+[1]*cos(x+[2])", -3.14159, 3.14159);
    f1->SetParLimits(0, 0, 1E7);
    f1->SetParLimits(1, 0, 1E7);
    f1->SetParLimits(2, 0, 1E7);
    h_clusPhi_incl->Fit("f1", "R P M N 0 Q");

    // Compute fit results
    float chi2ndf = f1->GetChisquare() / f1->GetNDF();
    float scaledchi2ndf = chi2ndf / f1->GetParameter(0);
    float BoverA = f1->GetParameter(1) / f1->GetParameter(0);
    float avgclus = f1->GetParameter(0);

    // Normalize occupancy histogram
    h_occupancy->Scale(1. / h_occupancy->Integral(-1, -1));
    h_occupancy->GetXaxis()->SetRange(1, h_occupancy->GetNbinsX() + 1);

    // Draw cluster phi histogram with fit
    canvas->cd(1);
    h_clusPhi_incl->SetTitle("Cluster #phi Distribution");
    h_clusPhi_incl->SetXTitle("Cluster #phi [rad]");
    h_clusPhi_incl->SetYTitle("Entries");
    h_clusPhi_incl->SetStats(0);
    h_clusPhi_incl->Draw();
    f1->SetLineColor(kRed);
    f1->Draw("same");

    // Draw chip occupancy histogram
    canvas->cd(2);
    h_occupancy->SetTitle("Chip Occupancy");
    h_occupancy->SetXTitle("Chip Occupancy [%]");
    h_occupancy->SetYTitle("Normalized Entries");
    h_occupancy->SetStats(0);
    h_occupancy->Draw();
    gPad->SetLogy();

    // Display summary info
    canvas->cd(3);
    TLatex latex;
    latex.SetTextSize(0.04);
    latex.DrawLatex(0.1, 0.8, Form("#chi^{2}/ndf: %.5g", chi2ndf));
    latex.DrawLatex(0.1, 0.7, Form("Scaled #chi^{2}/ndf: %.5g", scaledchi2ndf));
    latex.DrawLatex(0.1, 0.6, Form("B/A: %.5g", BoverA));
    latex.DrawLatex(0.1, 0.5, Form("Avg Clusters: %.5g", avgclus));

    // Print run details
    canvas->cd(4);
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.04);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);
    std::ostringstream runnostream;
    //runnostream << "MVTX Info Run " << cl->RunNumber() << ", build " << cl->build();
    PrintRun.DrawText(0.5, 0.5, runnostream.str().c_str());

    // Update and save canvas
    canvas->Update();
    //canvas->SaveAs("MVTX_Summary.pdf");
    std::cout << "MVTX DrawSummaryInfo() Ending" << std::endl;
}


In [18]:
%%cpp 
MVTXDrawSummaryInfo(qa_file_new);

MVTX DrawSummaryInfo() Beginning
MVTX DrawSummaryInfo() Ending


In [19]:
%%cpp -d

void MVTXRawHit(TFile* qa_file_new) {
    std::cout << "MVTX RawHitInfo() Beginning" << std::endl;

    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return;
    }
     
    std::string histprefix = "h_MvtxRawHitQA_"; 

    // Retrieve histograms for DrawChipInfo

    TH1* h_nhits_layer0 = dynamic_cast<TH1 *>(qa_file_new->Get((histprefix + "nhits_layer0").c_str()));
    TH1* h_nhits_layer1 = dynamic_cast<TH1 *>(qa_file_new->Get((histprefix + "nhits_layer1").c_str()));
    TH1* h_nhits_layer2 = dynamic_cast<TH1 *>(qa_file_new->Get((histprefix + "nhits_layer2").c_str()));
    TH1* h_bco = dynamic_cast<TH1 *>(qa_file_new->Get((histprefix + "bco").c_str()));
    TH1* h_strobe_bc = dynamic_cast<TH1 *>(qa_file_new->Get((histprefix + "strobe_bc").c_str()));
    TH1* h_chip_bc = dynamic_cast<TH1 *>(qa_file_new->Get((histprefix + "chip_bc").c_str()));


    // Create canvas
    TCanvas *canvas = new TCanvas("summary_info", "MVTX Summary Info", 1000, 800);
    canvas->Divide(2, 3);

    // Draw hit layers
    canvas->cd(1);
    if (h_nhits_layer0 && h_nhits_layer1 && h_nhits_layer2) {
        h_nhits_layer0->SetTitle("MVTX Raw Hit Number");
        h_nhits_layer0->SetXTitle("Hit number");
        h_nhits_layer0->SetYTitle("Entries");
        h_nhits_layer0->SetMarkerColor(kRed);
        h_nhits_layer0->SetLineColor(kRed);
        auto ymax = std::max({h_nhits_layer0->GetMaximum(), h_nhits_layer1->GetMaximum(), h_nhits_layer2->GetMaximum()});
        h_nhits_layer0->SetMaximum(1.2 * ymax);
        h_nhits_layer0->DrawCopy();
        h_nhits_layer1->SetMarkerColor(kBlue);
        h_nhits_layer1->SetLineColor(kBlue);
        h_nhits_layer1->DrawCopy("same");
        h_nhits_layer2->SetMarkerColor(kBlack);
        h_nhits_layer2->SetLineColor(kBlack);
        h_nhits_layer2->DrawCopy("same");
        auto legend = new TLegend(0.55, 0.7, 0.83, 0.9);
        legend->AddEntry(h_nhits_layer0, "Layer0", "pl");
        legend->AddEntry(h_nhits_layer1, "Layer1", "pl");
        legend->AddEntry(h_nhits_layer2, "Layer2", "pl");
        legend->Draw("same");
        gPad->SetLogy();
    }

    // Draw BCO
    canvas->cd(2);
    if (h_bco) {
        h_bco->SetTitle("MVTX BCO");
        h_bco->DrawCopy();
        gPad->SetRightMargin(0.15);
    }

    // Draw Strobe BC
    canvas->cd(3);
    if (h_strobe_bc) {
        h_strobe_bc->SetTitle("MVTX Strobe BC");
        h_strobe_bc->DrawCopy();
        gPad->SetRightMargin(0.15);
    }

    // Draw Chip BC
    canvas->cd(4);
    if (h_chip_bc) {
        h_chip_bc->SetTitle("MVTX Chip BC");
        h_chip_bc->DrawCopy();
        gPad->SetRightMargin(0.15);
    }

    // Display summary info
    canvas->cd(5);
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.04);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);
    std::ostringstream runnostream;
    //runnostream << "MVTX Info Run " << cl->RunNumber() << ", build " << cl->build();
    PrintRun.DrawText(0.5, 0.5, runnostream.str().c_str());

    // Update and save canvas
    canvas->Update();
    std::cout << "MVTX DrawSummaryInfo() Ending" << std::endl;
}


In [20]:
%%cpp 
MVTXRawHit(qa_file_new);

MVTX RawHitInfo() Beginning
MVTX DrawSummaryInfo() Ending


Warning in <TCanvas::Constructor>: Deleting canvas with same name: summary_info


In [31]:
%%cpp

int MVTXDrawHitMapInfo(TFile* qa_file_new) {
  std::cout << "MVTX DrawHitMapInfo() Beginning" << std::endl;

    // Check if the ROOT file is valid
    if (!qa_file_new || qa_file_new->IsZombie()) {
        std::cerr << "Error opening file" << std::endl;
        return -1;
    }

    std::string histprefix = "h_MvtxRawHitQA_"; 

  TH2* h_nhits_stave_chip_layer0 = dynamic_cast<TH2*>(qa_file_new->Get((histprefix + "nhits_stave_chip_layer0").c_str()));
  TH2* h_nhits_stave_chip_layer1 = dynamic_cast<TH2*>(qa_file_new->Get((histprefix + "nhits_stave_chip_layer1").c_str()));
  TH2* h_nhits_stave_chip_layer2 = dynamic_cast<TH2*>(qa_file_new->Get((histprefix + "nhits_stave_chip_layer2").c_str()));

  // Create canvas and divide into 5 pads
  TCanvas* canvas = new TCanvas("hitmap_info", "MVTX Hit Map Info", 1800, 1000);
  canvas->Divide(5, 1); // 5 rows, 1 column

  // Pad 1: Hit numbers for each layer
  canvas->cd(1);
  if (h_nhits_stave_chip_layer0 && h_nhits_stave_chip_layer1 && h_nhits_stave_chip_layer2) {
    TH1* h_nhits_chip_layer0 = h_nhits_stave_chip_layer0->ProjectionX();
    TH1* h_nhits_chip_layer1 = h_nhits_stave_chip_layer1->ProjectionX();
    TH1* h_nhits_chip_layer2 = h_nhits_stave_chip_layer2->ProjectionX();
    
    h_nhits_chip_layer0->SetTitle("MVTX Raw Hit Number vs. ChipID");
    h_nhits_chip_layer0->SetXTitle("ChipID");
    h_nhits_chip_layer0->SetYTitle("Entries");
    h_nhits_chip_layer0->SetMarkerColor(kRed);
    h_nhits_chip_layer0->SetLineColor(kRed);
    
    auto ymax_l0 = h_nhits_chip_layer0->GetMaximum();
    auto ymax_l1 = h_nhits_chip_layer1->GetMaximum();
    auto ymax_l2 = h_nhits_chip_layer2->GetMaximum();
    
    h_nhits_chip_layer0->SetMaximum(1.2 * std::max({ymax_l0, ymax_l1, ymax_l2}));
    h_nhits_chip_layer0->SetMinimum(0);
    h_nhits_chip_layer0->DrawCopy();
    
    h_nhits_chip_layer1->SetMarkerColor(kBlue);
    h_nhits_chip_layer1->SetLineColor(kBlue);
    h_nhits_chip_layer1->DrawCopy("same");
    
    h_nhits_chip_layer2->SetMarkerColor(kBlack);
    h_nhits_chip_layer2->SetLineColor(kBlack);
    h_nhits_chip_layer2->DrawCopy("same");
    
    auto legend = new TLegend(0.55, 0.2, 0.83, 0.4);
    legend->AddEntry(h_nhits_chip_layer0, "Layer0", "pl");
    legend->AddEntry(h_nhits_chip_layer1, "Layer1", "pl");
    legend->AddEntry(h_nhits_chip_layer2, "Layer2", "pl");
    legend->Draw("same");
    
    gPad->SetRightMargin(0.15);
  } else {
    return -1;
  }

  // Pad 2: Hit numbers vs. StaveID
  canvas->cd(2);
  if (h_nhits_stave_chip_layer0 && h_nhits_stave_chip_layer1 && h_nhits_stave_chip_layer2) {
    TH1* h_nhits_stave_layer0 = h_nhits_stave_chip_layer0->ProjectionY();
    TH1* h_nhits_stave_layer1 = h_nhits_stave_chip_layer1->ProjectionY();
    TH1* h_nhits_stave_layer2 = h_nhits_stave_chip_layer2->ProjectionY();
    
    h_nhits_stave_layer2->SetTitle("MVTX Raw Hit Number vs. StaveID");
    h_nhits_stave_layer2->SetXTitle("StaveID");
    h_nhits_stave_layer2->SetYTitle("Entries");
    h_nhits_stave_layer2->SetMarkerColor(kBlack);
    h_nhits_stave_layer2->SetLineColor(kBlack);
    
    auto ymax_l0 = h_nhits_stave_layer0->GetMaximum();
    auto ymax_l1 = h_nhits_stave_layer1->GetMaximum();
    auto ymax_l2 = h_nhits_stave_layer2->GetMaximum();
    
    h_nhits_stave_layer2->SetMaximum(1.2 * std::max({ymax_l0, ymax_l1, ymax_l2}));
    h_nhits_stave_layer2->SetMinimum(0);
    h_nhits_stave_layer2->DrawCopy("HIST");
    
    h_nhits_stave_layer1->SetMarkerColor(kBlue);
    h_nhits_stave_layer1->SetLineColor(kBlue);
    h_nhits_stave_layer1->DrawCopy("HIST,same");
    
    h_nhits_stave_layer0->SetMarkerColor(kRed);
    h_nhits_stave_layer0->SetLineColor(kRed);
    h_nhits_stave_layer0->DrawCopy("HIST,same");
    
    auto legend = new TLegend(0.55, 0.7, 0.83, 0.9);
    legend->AddEntry(h_nhits_stave_layer0, "Layer0", "pl");
    legend->AddEntry(h_nhits_stave_layer1, "Layer1", "pl");
    legend->AddEntry(h_nhits_stave_layer2, "Layer2", "pl");
    legend->Draw("same");
    
    gPad->SetRightMargin(0.15);
  } else {
    return -1;
  }

  // Pad 3: Hit map for layer 0
  canvas->cd(3);
  if (h_nhits_stave_chip_layer0) {
    h_nhits_stave_chip_layer0->SetTitle("MVTX Raw Hit Map Layer 0");
    h_nhits_stave_chip_layer0->SetXTitle("Chip ID");
    h_nhits_stave_chip_layer0->SetYTitle("Stave ID");
    h_nhits_stave_chip_layer0->DrawCopy("colz");
    gPad->SetRightMargin(0.15);
  } else {
    return -1;
  }

  // Pad 4: Hit map for layer 1
  canvas->cd(4);
  if (h_nhits_stave_chip_layer1) {
    h_nhits_stave_chip_layer1->SetTitle("MVTX Raw Hit Map Layer 1");
    h_nhits_stave_chip_layer1->SetXTitle("Chip ID");
    h_nhits_stave_chip_layer1->SetYTitle("Stave ID");
    h_nhits_stave_chip_layer1->DrawCopy("colz");
    gPad->SetRightMargin(0.15);
  } else {
    return -1;
  }

  // Pad 5: Hit map for layer 2
  canvas->cd(5);
  if (h_nhits_stave_chip_layer2) {
    h_nhits_stave_chip_layer2->SetTitle("MVTX Raw Hit Map Layer 2");
    h_nhits_stave_chip_layer2->SetXTitle("Chip ID");
    h_nhits_stave_chip_layer2->SetYTitle("Stave ID");
    h_nhits_stave_chip_layer2->DrawCopy("colz");
    gPad->SetRightMargin(0.15);
  } else {
    return -1;
  }

  // Add a text with run information
  TText PrintRun;
  PrintRun.SetTextFont(62);
  PrintRun.SetTextSize(0.04);
  PrintRun.SetNDC();
  PrintRun.SetTextAlign(23);
  std::ostringstream runnostream1;
  std::string runstring1;
  //runnostream1 << Name() << "_mvtx HitMap Info Run " << cl->RunNumber() << ", build " << cl->build();
  runstring1 = runnostream1.str();
  //transparent[1]->cd();
  PrintRun.DrawText(0.5, 1., runstring1.c_str());

  canvas->Update();

  std::cout << "DrawHitMapInfo Ending" << std::endl;
  return 0;
}


In [32]:
%%cpp 
MVTXDrawHitMapInfo(qa_file_new);

MVTX DrawHitMapInfo() Beginning
DrawHitMapInfo Ending


Warning in <TCanvas::Constructor>: Deleting canvas with same name: hitmap_info
